In [1]:
import platform, sys    
print(platform.python_implementation()) #python interpreter
print(sys.version) #python version

CPython
3.10.9 (tags/v3.10.9:1dd9be6, Dec  6 2022, 20:01:21) [MSC v.1934 64 bit (AMD64)]


In [2]:
import numpy as np
from scipy.stats import norm

# The basic salvo combat model
as per: Michael J Armstrong (2014) The salvo combat model with a sequential
exchange of fire, Journal of the Operational Research Society, 65:10, 1593-1601, DOI: 10.1057/
jors.2013.115


In [3]:
# battle model parameters
class PlayerState():
    def __init__(self, N, offPower, defPower, stayPower):
        self.N = N  # number of units
        # number of missiles (accurately) fired per unit per salvo
        self.offPower = offPower
        self.defPower = defPower  # number of missiles intercepted per unit per salvo
        # staying power, how many hits a unit can take from opponent.
        self.stayPower = stayPower
        return (None)


class Game():
    def __init__(self, red: PlayerState, blue: PlayerState):
        self.red = red
        self.blue = blue
        return (None)

    def battle(self):

        # helper function to get lost number of units each round of battle.

        def get_delta_N(P1: PlayerState, P2: PlayerState):
            deltaP1 = (P2.offPower*P2.N - P1.defPower*P1.N)*(1/P1.stayPower)
            deltaP1 = max(0, min(deltaP1, P1.N))
            return (deltaP1)

        # loop while both players still have units
        it = 0
        while (self.red.N > 0 and self.blue.N > 0 and it < 50):
            

            red_delta_N = get_delta_N(self.red, self.blue)
            blue_delta_N = get_delta_N(self.blue, self.red)
               
            self.red.N = self.red.N - red_delta_N         
            self.blue.N = self.blue.N - blue_delta_N
            it +=1  
            print(f"Nred :{self.red.N}, Nblue : {self.blue.N} ")
            
        return (self)


In [12]:
blue_player = PlayerState(N=6, offPower=4, defPower=2, stayPower=5)
red_player = PlayerState(N=4, offPower=5, defPower=3, stayPower=4)

game = Game(blue_player, red_player)
game = game.battle()


Nred :4.4, Nblue : 1.0 
Nred :4.4, Nblue : 0.0 


# Sequential Stochastic salvo combat model 
as per: Michael J Armstrong (2014) The salvo combat model with a sequential
exchange of fire, Journal of the Operational Research Society, 65:10, 1593-1601, DOI: 10.1057/
jors.2013.115


In [16]:
# battle model parameters
class PlayerState():
    def __init__(self, N=5, offNumber=8, offProb=2/3, offDamage=1/3,  defNumber=4, defProb=1/4, stayPower=3):
        self.N = N  # number of units
        self.offNumber = offNumber
        self.offProb = offProb
        self.offDamage = offDamage

        self.defNumber = defNumber
        self.defProb = defProb
        self.stayPower = stayPower
        
        return (None)


class Game():
    def __init__(self, red: PlayerState, blue: PlayerState):
        self.red = red
        self.blue = blue
        return (None)

    def battle(self):

        # helper function to get lost number of units each round of battle.
        # P2 attacking P1
        def getPlayerN(P1: PlayerState, P2: PlayerState):

            mu_p2_off = P2.N*P2.offNumber*P2.offProb
            var_p2_off = P2.N * P2.offNumber * P2.offProb*(1-P2.offProb)

            mu_p1_def = P1.N*P1.defNumber*P1.defProb
            var_p1_def = P1.N * P1.defNumber*P1.defProb*(1-P1.defProb)

            # draw from normal distribution
            netOffHits = np.random.normal( 
                loc=(P2.N*mu_p2_off-P1.N*mu_p1_def),
                scale=np.sqrt((P2.N*var_p2_off + P1.N*var_p1_def))
            )
            varDamage = 1/(2.5*P2.offDamage)
            
            mu_P1_N = P1.N -(mu_p2_off - mu_p1_def)*P2.offDamage
            var_P1_N =  (mu_p2_off - mu_p1_def)*varDamage + (var_p2_off + var_p1_def)*P2.offDamage**2 \
                # - 2*varDamage*(mu_p2_off - mu_p1_def)*norm.cdf(0,loc=mu_P1_N, scale=var_P1_N) \
                # + 2*varDamage*(var_p2_off + var_p1_def)*norm.pdf(0,loc=mu_P1_N, scale=var_P1_N)
            
            
            return (max(0,norm.rvs(loc=mu_P1_N, scale=np.sqrt(var_P1_N), size=1)))

        # loop while both players still have units
        it = 0
        while (self.red.N > 0 and self.blue.N > 0 and it < 50):

            #simultaneous attack
            red_new_N = getPlayerN(self.red, self.blue)
            blue_new_N = getPlayerN(self.blue, self.red)

            self.red.N = red_new_N
            self.blue.N = blue_new_N
            it += 1
            print(f"Nred :{self.red.N}, Nblue : {self.blue.N} ")

        return (self)


In [18]:
blue_player = PlayerState(N=15,defNumber=8)
red_player = PlayerState(N=15, defNumber=8)

for _ in range(100):
    game = Game(blue_player, red_player)
    game = game.battle()


Nred :[2.23890853], Nblue : [19.50832798] 


C:\Users\IsakG\AppData\Local\Temp\ipykernel_22588\1746868981.py:47: RuntimeWarning: invalid value encountered in sqrt
  return (max(0,norm.rvs(loc=mu_P1_N, scale=np.sqrt(var_P1_N), size=1)))


ValueError: Domain error in arguments. The `scale` parameter must be positive for all distributions, and many distributions have restrictions on shape parameters. Please see the `scipy.stats.norm` documentation for details.